
## Analyse automatique du narratif de corpus
`Shami THIRION SEN`

[Lien vers le git du projet](https://github.com/sham-glam/semantique)

### Introduction

Après avoir fait une première analyse du narratif du communisme au premier semestre, l'object du projet actuel est de mettre en pratique l'analyse automatique du narratif avec les outils TAL. Les étapes principales pour mener cette analyse sont la constitution du corpus, un traitement efficace et une modélisation qui puisser « *faire parler* » le narratif. Les approches automatique employées sont: <br>
**1. le topic modelling avec LDA gensim** <br>
**2. le calcul de spécificité des corpus avec le package *Textometry* en R**<br>

Faire parler / constituer un corpus qui puisse // est une méthode incertain et difficile à définir. D'où l'intérêt du présent projet de de constituer un **corpus constrastif** ou on peut repérer des «termes» qui n'existent pas dans l'autre[<sup id="fn1-back">1</sup>].

N'ayant pas de traits particulier qui définissent un corpus qui puisse révéler ..





[conclude intro] 


### Constitution et descriptif du corpus

La constitution du corpus a été la première étape, et la plus complexe. Afin d'illuster ? le narratif du communisme au bengal, l'objectif idéal était l'observation d'un corpus contemporain, issu de la presse du bengal de l'ouest. Or, la digitalisation de la presse écrite au bengal est un phénomène plus récent. Ainsi, nous avons décidé d'élargir la période sur laquelle nous allons extraire le corpus. Ceci nous dévie un peu, mais en ce qui concerne le TAL, ça facilite la tâche. [ Time]

La deuxième complexité consiste en définition ce qu'est un corpus constrastif. Si 


Ainsi, [A ma grand surprise], 

La source principale, qui a permis une extraction «massive» des données est [Internet Archives](https://web.archive.org/). 
- construction de corpus contrastif
  - corpus communiste vs. «non-communiste»
 
Pour le corpus (qu'on souhaite que soit représentatif du narratif communiste  est issu de *Ganashakti*
Il y a deux sites [ganashakti.com](https://www.ganashakti.com/) renvoie au site les 


- Bartamanpatrika

- Anandabazar



### Scripts et automatisation des tâches
Le plus gros défi et le plus grand a été la constitution du corpus. Le but étant une étude comparatif de corpus « contrastif »

#### La puissance de TAL !


### Semantique dans tout ça...



- défi temps - disponibilité des données 
  - si le tp sem 1 visait  le narratifs des temps communiste, (en hind sight) l'obtention du corpus est un défi non négligeable. À l'époque le Bengal est encore loin de la digitalisation des médias.
  

- 

# Semantique textuelle



- construction de corpus contrastif
  - corpus communiste vs. «non-communiste»

- défi temps - disponibilité des données 
  - si le tp sem 1 visait  le narratifs des temps communiste, (en hind sight) l'obtention du corpus est un défi non négligeable. À l'époque le Bengal est encore loin de la digitalisation des médias.
  
[<sup id="fn2">1</sup>](#fn1-back)*However, psychological work on categorisation suggests that cognitive categories are not usually "hard and fast" but instead have fuzzy boundaries, so it is not so much a question of whether an item belongs to one category or the other, but how often it falls into one category as opposed to the other one.*



In [1]:
import numpy as np
import json
import glob
from pprint import pprint
from collections import defaultdict
import os, math
from utils import * # import des fonctions depuis le fichier utils

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#visualisation
import pyLDAvis
import pyLDAvis.gensim

# outils TAL en bengali
import BnLemma as lm
from bnlp import BengaliPOS

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Bangla POS tagging
bn_pos = BengaliPOS()
from bnlp import BengaliCorpus as corpus
stopwords = corpus.stopwords[:20] + ['রি' + 'টি']

In [3]:
def gen_words(texts):
    single_string = [ ' '.join(line) for line in texts]
    data_words = [gensim.utils.simple_preprocess(text) for text in single_string]
    return data_words


In [4]:
## BIGRAMMES ET TIGRAMMES
def make_bigrams(texts):
    # print(bigram[doc] for doc in texts)
    return ([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return([trigram[bigram[doc]] for doc in texts])


In [5]:
# on choisit le nombre de tokens (pour les 3 corpus)
num_tokens = 10000

#### BanglaGanashakti 
- Lecture du corpus extrait et nettoyage

In [6]:
banglaGanashakti = "../corpus/txtFiles/banglaGanashakti.txt"
filtered_ganashakti = read_corpus(banglaGanashakti, num_tokens)
filtered_ganashakti = [list for list in filtered_ganashakti if len(list)>0] # old list_of_docs


In [7]:
# generation   
data_words  = gen_words(filtered_ganashakti)
print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_ganashakti, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_ganashakti], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_ganashakti)
data_bigrams_trigrams = make_trigrams(data_bigrams)

# data_bigrams_trigrams = make_bigrams_trigrams(filtered_ganashakti)

['আরএসএস', 'অন', 'যবস', 'রহসন']


In [8]:
pprint(data_bigrams_trigrams[:3])

[['বিধান', 'গণতন্ত্র'], ['বিধান', 'প্রস্তাব', 'তন্ত্র', 'শব্দ', 'সরকার', 'বেসরকারি', 'হাত', 'পরিষ্কার'], ['বিয়োজন', 'চেয়ে', 'বিজেপি', 'সাংসদ', 'সরকার', 'মুখ', 'এমন', 'দেশ']]


In [9]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_ganashakti]

word = id2word[[9][:1][0]]
print (word)

এমন


In [10]:
corpus=get_corpus(corpus,id2word)
# corpus[:10]
# print(corpus)

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha="auto")

In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.295504 -0.061248       1        1  11.600992
14     0.263761  0.105912       2        1  10.624463
8     -0.261782  0.038802       3        1   7.313512
11    -0.240336 -0.115297       4        1   6.964433
13     0.050112 -0.247740       5        1   6.648896
3     -0.067546 -0.227225       6        1   6.390400
5     -0.152436 -0.192131       7        1   5.687671
1      0.170549  0.053327       8        1   5.061884
17    -0.009761  0.241741       9        1   4.629135
0      0.076874 -0.134812      10        1   4.397035
10     0.160965 -0.156159      11        1   4.145809
6     -0.091424  0.196344      12        1   3.930806
12     0.128589  0.150397      13        1   3.759416
18    -0.154470  0.120780      14        1   3.725783
2      0.047363  0.184121      15        1   3.647418
9     -0.045535  0.076044      16        1   2.709657
16    -0.137163  0.018289      17        1   2.461341
7     -0.026697  0.005879      18        1   2.270608
4      0.037012 -0.025101      19        1   2.206326
19    -0.043579 -0.031923      20        1   1.824414, topic_info=        Term      Freq     Total Category  logprob  loglift
11       দেশ  9.000000  9.000000  Default  30.0000  30.0000
296   সংস্থা  6.000000  6.000000  Default  29.0000  29.0000
333     তরুণ  7.000000  7.000000  Default  28.0000  28.0000
181      ডেল  2.000000  2.000000  Default  27.0000  27.0000
194      মাঠ  5.000000  5.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
16   ক্ষেত্র  0.020656  2.040246  Topic20  -6.3994  -0.5889
17     পুঁজি  0.020656  1.280016  Topic20  -6.3994  -0.1227
18    প্রবেশ  0.020656  1.280016  Topic20  -6.3994  -0.1227
19     বিলটি  0.020656  1.280016  Topic20  -6.3994  -0.1227
20     সংবাদ  0.020656  2.642853  Topic20  -6.3994  -0.8477

[830 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
156       1  0.276618     অংশ
156       4  0.276618     অংশ
213      16  0.915548  অথরিটি
116       1  0.499446  অধিকার
116       7  0.499446  অধিকার
...     ...       ...     ...
92        9  0.366974  হিন্দু
247      14  0.855303  হিমশিম
175       2  0.737591   হুগলী
352       8  0.819385       ে
297       6  0.785001    ১২টা

[547 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 15, 9, 12, 14, 4, 6, 2, 18, 1, 11, 7, 13, 19, 3, 10, 17, 8, 5, 20])

In [13]:
# pyLDAvis.save_html(vis, '../topic_modeling_ganashaki.html')

#### Calcul spécificité

In [14]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_ganashakti, 10)

In [15]:
print('Token les plus spécifiques:\n')
pprint(max_specificity)
print('\nToken les moins spécifiques:\n')
pprint(min_specificity)

Token les plus spécifiques:

{'আইন': 1.0,
 'উত্তর': -0.16490205818083653,
 'কথা': -0.16490205818083653,
 'কারন': -0.16490205818083653,
 'ঘটনা': 1.0,
 'দেশ': 1.0,
 'নাম': -0.16490205818083653,
 'প্রশ্ন': 1.0,
 'ভাঙা': -0.16490205818083653,
 'রাজ্য': -0.16490205818083653}

Token les moins spécifiques:

{'অংশ': -1.0220028753371122,
 'কাজ': -1.8781943549635112,
 'কৃষ্ণ': -1.8781943549635112,
 'প্রকট': -0.6276153592043174,
 'বধির': -1.4413673776561569,
 'বেসরকারি': -0.7366975313989659,
 'ব্যবস্হা': -0.6276153592043174,
 'মাঠ': -1.8781943549635112,
 'মাধ্যম': -0.6276153592043174,
 'সভা': -1.0220028753371122}


### Trying Top2vec

In [16]:
from top2vec import Top2Vec

2024-05-15 18:35:33.456496: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 18:35:33.459563: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-15 18:35:33.503238: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 18:35:34.474884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
ganashakti =  [' '.join(map(str, sublist)) for sublist in filtered_ganashakti]
print(len(filtered_ganashakti))
print(ganashakti[:5])
print(len(ganashakti))
# print(filtered_ganashakti[:5])
model = Top2Vec(ganashakti, min_count=2)

2024-05-15 18:35:37,409 - top2vec - INFO - Pre-processing documents for training
2024-05-15 18:35:37,416 - top2vec - INFO - Downloading universal-sentence-encoder-multilingual model


138
['বিধান গণতন্ত্র', 'বিধান প্রস্তাব তন্ত্র শব্দ সরকার বেসরকারি হাত পরিষ্কার', 'বিয়োজন চেয়ে বিজেপি সাংসদ সরকার মুখ এমন দেশ', 'সংসদ বেসরকারি বিলটি সংবাদ ক্ষেত্র দেশ পুঁজি প্রবেশ', 'মাধ্যম প্রতিবেদন স্বনির্ভরতায় প্রশ্ন চিহ্ন পড়া']
138


2024-05-15 18:35:39,099 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2024-05-15 18:35:40,102 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2024-05-15 18:35:47,544 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2024-05-15 18:35:47,550 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


In [18]:
topic_sizes, topic_nums = model.get_topic_sizes()

print(topic_sizes)

[138]


In [19]:
topic_words, word_scores, topic_nums = model.get_topics(1)

In [20]:
for words, scores, num in zip(topic_words, word_scores, topic_nums):
    print(num)
    print(f'words = {words}')

0
words = ['পরশ' 'পদ' 'আয' 'ইন' 'কষণ' 'গজ' 'ধযম' 'ধরম' 'পর' 'লত' 'রপ' 'মখ' 'মক'
 'পরশন' 'পল' 'সভ' 'সরক' 'বছর' 'মপ' 'বধ' 'রত' 'রট' 'শর' 'যয' 'সথ' 'সদ'
 'বযবসহ' 'সতর' 'আকরমণ' 'ওয' 'কর' 'করম' 'আইন' 'অভ' 'কষ' 'কষক' 'তর' 'গর'
 'ঘটন' 'জম' 'তরণ' 'নত' 'নদ' 'নষ' 'পণত' 'পরক' 'পরকট' 'পরত' 'পরযটক' 'অনল']


### Bartamanpatrika

In [21]:
bartamanpatrika = "../corpus/txtFiles/bartamanpatrika.txt"
filtered_bartamanpatrika = read_corpus(bartamanpatrika, num_tokens)
filtered_bartamanpatrika = [list for list in filtered_bartamanpatrika if len(list)>0]

In [22]:
# generation   
data_words  = gen_words(filtered_bartamanpatrika)
# print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_bartamanpatrika, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_bartamanpatrika], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_bartamanpatrika)
data_bigrams_trigrams = make_trigrams(data_bigrams)


In [23]:
print(data_bigrams_trigrams[:10])

[['শত', 'নেতৃত্ব', 'ক্রিকেট', 'কন্ট্রোল', 'বোর্ড', 'প্রস্তাব', 'গ্রহণ', 'শত', 'টেস্ট', 'অধিনায়ক', 'ম্যাচ', 'মাইলস্টোন', 'ম্যাচ', 'চাওয়া', 'অভিমানী', 'কে'], ['টেস্ট', 'অধিনায়ক', 'কোকিল', 'ইস্তফা', 'পাঁচ', 'বুমরাহকেও'], ['হ্যামস্ট্রিংয়ে', 'চোট', 'ওয়েটিং', 'ইন্ডিজের', 'বিরুদ্ধ', 'সিরিজ', 'শর্করা', 'ফেব্রুয়ারি', 'গোড়ায়', 'ভারত'], ['অধিনায়ক', 'অধীনে', 'কাজ'], ['ের', 'ব্যালট', 'ডিও', 'সি', 'ল্যাং', 'বা', 'পায়ে', 'জাদু', 'হার'], ['ভ্যাকসিন', 'নেব', 'নোভাক', 'জকোভিচ', 'প্রশ্ন', 'বিশ্ব'], ['ভার', 'গ্যালারি', 'নোনোভাক', 'লেখা', 'পোস্ট', 'জকোভিচ', 'অস্ট্রেলিয়ান'], ['প্রত্যাখ্যাত', 'বিলাত', 'গোলে', 'সুপার', 'কাপ', 'ঘর', 'রিয়াল', 'মাদ্রাসা', 'রবিবার', 'সৌদি', 'আরব'], ['আইএসএলে', 'রানার', 'মোহন', 'বাগান', 'সুবাদ', 'ব্রিগেড', 'কাপের', 'প্লেঅফে'], ['আইএসএলে', 'সোমবার', 'জাম', 'হায়', 'এফ', 'ম্যাচ']]


In [24]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_bartamanpatrika]

word = id2word[[9][:1][0]]
print (word)

প্রস্তাব


In [25]:
corpus=get_corpus(corpus,id2word)


In [26]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")



In [27]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
22    -0.262946 -0.187550       1        1  12.303715
23     0.110805 -0.283114       2        1   7.686677
5     -0.270381  0.167367       3        1   6.213219
0      0.285076 -0.104281       4        1   6.009286
24     0.266567  0.000686       5        1   5.685919
20     0.249581  0.148460       6        1   5.002748
17     0.149744  0.202527       7        1   4.006785
28    -0.031248 -0.237959       8        1   3.959544
10    -0.199417 -0.118107       9        1   3.829569
16    -0.092495 -0.199780      10        1   3.758769
19    -0.234652  0.035064      11        1   3.700511
8     -0.130587  0.223928      12        1   3.454443
15     0.185471 -0.152506      13        1   3.395673
7     -0.091039 -0.121979      14        1   3.148744
26     0.046371  0.229246      15        1   3.147210
27    -0.123175  0.124002      16        1   3.089252
2     -0.026518  0.235538      17        1   3.023498
3     -0.160501 -0.033687      18        1   2.896905
11     0.065275 -0.166942      19        1   2.718304
1      0.101597 -0.078380      20        1   2.524724
12     0.144019  0.057368      21        1   2.042682
21     0.053054  0.137646      22        1   1.847475
18    -0.091348  0.044038      23        1   1.720603
25    -0.030582 -0.044057      24        1   1.291777
9     -0.009660  0.041854      25        1   0.920524
6      0.043422  0.049212      26        1   0.918281
13     0.033244  0.008453      27        1   0.671687
4      0.009502 -0.002912      28        1   0.550258
14     0.005411  0.012933      29        1   0.240611
29     0.005411  0.012933      30        1   0.240611, topic_info=               Term       Freq      Total Category  logprob  loglift
98          ব্যবসায়  13.000000  13.000000  Default  30.0000  30.0000
90              যোগ  10.000000  10.000000  Default  29.0000  29.0000
97            বৃদ্ধ   9.000000   9.000000  Default  28.0000  28.0000
88         প্রাপ্তি   5.000000   5.000000  Default  27.0000  27.0000
105          সাফল্য   7.000000   7.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
25           শর্করা   0.003763   1.105916  Topic30  -6.2324   0.3465
26            সিরিজ   0.003763   3.194495  Topic30  -6.2324  -0.7143
27   হ্যামস্ট্রিংয়ে   0.003763   1.105916  Topic30  -6.2324   0.3465
28            অধীনে   0.003763   1.274534  Topic30  -6.2324   0.2046
29              কাজ   0.003763   3.092308  Topic30  -6.2324  -0.6818

[1291 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
251       1  0.498160             অংশ
251       8  0.498160             অংশ
233       6  0.817674       অক্সফোর্ড
93        5  0.179165          অগ্রসর
93        8  0.179165          অগ্রসর
...     ...       ...             ...
27       19  0.904228  হ্যামস্ট্রিংয়ে
329       6  0.817674             াদি
161      16  0.891302               ে
38       13  0.868978              ের
148       9  0.853596              য়ে

[660 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 24, 6, 1, 25, 21, 18, 29, 11, 17, 20, 9, 16, 8, 27, 28, 3, 4, 12, 2, 13, 22, 19, 26, 10, 7, 14, 5, 15, 30])

In [28]:
pyLDAvis.save_html(vis, '../topic_modeling_bartamanpatrika.html')

#### Calcul de spécificité

In [29]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_bartamanpatrika, 10)
# max_specificity, min_specificity

In [30]:
print('Token les plus spécifiquess:\n')
pprint(max_specificity)
print('\nToken les moins spécifiques:\n')
pprint(min_specificity)

Token les plus spécifiquess:

{'কর্ম': 1.0,
 'কর্র': 1.0,
 'গ্রহণ': 0.6578140741920342,
 'প্রসার': 1.0,
 'প্রাপ্তি': 0.8228549743404974,
 'পড়ুয়া': 1.0,
 'ম্যাচ': 1.0,
 'সিরিজ': 0.6578140741920342,
 'সুখ': 1.0,
 'স্কুল': 1.0}

Token les moins spécifiques:

{'অগ্রসর': -0.39592813802188326,
 'উপার্জন': -1.3072331639039008,
 'বাধা': -0.2629433143375571,
 'বিদ্যা': -0.5558956879452186,
 'ব্যবসায়': -0.5497624422193942,
 'ভারত': -0.2629433143375571,
 'মন': -0.44497511669400514,
 'যোগ': -0.23433628731623313,
 'শান্তি': -0.2629433143375571,
 'সাফল্য': -0.24645547017492428}


### Anandabazar

- corpus qui avec 1000 tokens pointe vers cricket et le sujet de sport 
- Alors qu'avec 10000 nous obtenons plus de généralité

In [31]:
anandabazar = "../corpus/txtFiles/anandabazar.txt"
filtered_anandabazar = read_corpus(anandabazar, num_tokens)
filtered_anandabazar = [list for list in filtered_anandabazar if len(list)>0] 

In [32]:
# generation   
data_words  = gen_words(filtered_anandabazar)
# print(data_words[9][:200])

# copy_this
bigram_phrases = gensim.models.Phrases(filtered_anandabazar, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[filtered_anandabazar], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

data_bigrams = make_bigrams(filtered_anandabazar)
data_bigrams_trigrams = make_trigrams(data_bigrams)


In [33]:
print(data_bigrams_trigrams[:10])

[['১৫টা', 'লোক', 'ঘিরে'], ['ষষ্ঠী', 'সপ্তম', 'তুলনা', 'রবিবার', 'গা', 'রাস্তা', 'গা', 'উত্তর', 'দক্ষিণ', 'নিয়ন্ত্রণ', 'বড়', 'পুজোগুলোয়', 'চোখ', 'পড়া'], ['রাজ্য', 'বিজড়িত', 'ভাঙা', 'দশ', 'রাম', 'সুনীল', 'বনসাই', 'দল', 'নেতৃত্ব', 'পর্যবেক্ষণ', 'মঙ্গল', 'পরিকল্পনা', 'পালন', 'দায়িত্ব'], ['কোটি', 'গুটখায়', 'রঙ', 'শহর'], ['শীর্ষ', 'আদালত', 'মানুষ', 'আস্থা', 'বজায়', 'দায়িত্ব', 'কার'], ['সাল', 'বিধান', 'নির্বচন', 'শুরু', 'বিজড়িত', 'সাল', 'বিতর্ক', 'আরাধন', 'সুকান্ত', 'বারেক', 'পুজোতেই', 'সুলতান'], ['কংগ্রেস', 'প্রস্তাব', 'বাদ', 'শুরু', 'জোট'], ['ডিএ', 'মামলা', 'ধাক্কা', 'খেল', 'রাজ্য', 'সরকার'], ['সোমবার', 'রাত', '৮টা', 'মৃত্যু', 'মৃত', 'পরিবার', 'দেহ', 'নিতে', 'স্বীকার'], ['বারেক', 'চেষ্টায়', 'দিন', 'যোগ', 'সরকার', 'হা', 'অনুমোদন', 'রাহুলপ্রিয়ঙ্কা']]


In [34]:
### TF-IDF

id2word = Dictionary(data_bigrams_trigrams)
corpus = [id2word.doc2bow(text) for text in filtered_anandabazar]

word = id2word[[9][:1][0]]
print (word)

corpus=get_corpus(corpus,id2word)
# corpus[:10]

পড়া


In [35]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha="auto")



In [36]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17     0.205128  0.192246       1        1  11.883552
9      0.054320 -0.269577       2        1  11.270144
8     -0.226120 -0.161464       3        1   8.565282
0     -0.126224  0.234438       4        1   8.314973
2      0.177095 -0.179443       5        1   6.861030
10     0.246002 -0.049098       6        1   6.687183
12     0.037122  0.236645       7        1   6.264594
1     -0.085719 -0.195909       8        1   4.783244
11    -0.223665  0.074913       9        1   4.707002
16     0.184272  0.068031      10        1   4.195423
15     0.119653  0.007497      11        1   3.671375
19     0.010861  0.138802      12        1   3.323449
13    -0.161195 -0.008783      13        1   3.078562
14    -0.100371  0.101330      14        1   2.820311
6      0.045384 -0.110775      15        1   2.808711
4     -0.059296  0.047954      16        1   2.708033
18    -0.027238 -0.078638      17        1   2.689293
3     -0.098684 -0.060521      18        1   2.557300
5      0.034629  0.009749      19        1   2.199686
7     -0.005956  0.002603      20        1   0.610853, topic_info=              Term      Freq     Total Category  logprob  loglift
154            ভোট  5.000000  5.000000  Default  30.0000  30.0000
93            বৈঠক  3.000000  3.000000  Default  29.0000  29.0000
198  প্রধানমন্ত্রী  3.000000  3.000000  Default  28.0000  28.0000
79             বার  3.000000  3.000000  Default  27.0000  27.0000
90           বাড়ি  3.000000  3.000000  Default  26.0000  26.0000
..             ...       ...       ...      ...      ...      ...
10       পুজোগুলোয়  0.008145  1.114701  Topic20  -6.3483   0.1791
11             বড়  0.008145  1.114701  Topic20  -6.3483   0.1791
12          রবিবার  0.008145  2.447346  Topic20  -6.3483  -0.6073
13          রাস্তা  0.008145  2.412150  Topic20  -6.3483  -0.5928
14           ষষ্ঠী  0.008145  1.114701  Topic20  -6.3483   0.1791

[848 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
466       4  0.596689    অনলাইন
68        3  0.609041   অনুমোদন
231       3  0.807175    অফিসার
379       2  0.778530    অবসরের
487       1  0.770290   অবস্থান
...     ...       ...       ...
313       7  0.328525  হাসপাতাল
313       9  0.328525  হাসপাতাল
249      12  0.977173       য়াদ
2         9  0.897103      ১৫টা
159       4  0.810579      ২৫টা

[516 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 10, 9, 1, 3, 11, 13, 2, 12, 17, 16, 20, 14, 15, 7, 5, 19, 4, 6, 8])

In [37]:
pyLDAvis.save_html(vis, '../topic_modeling_anandabazar.html')

#### Calcul de spécificité

In [38]:
max_specificity, min_specificity = get_higest_lowest_specificity(filtered_anandabazar, 10)

In [39]:
print('Token les plus spécifiques:\n')
pprint(max_specificity)
print('\nToken les moins spécifiques:\n')
pprint(min_specificity)

Token les plus spécifiques:

{'গা': 1.0,
 'দায়িত্ব': 1.0,
 'দিন': 1.0,
 'নেতা': 1.0,
 'পরিবার': 1.0,
 'বিধান': 1.0,
 'মৃত্যু': 1.0,
 'রবিবার': 1.0,
 'রাজ্য': 1.0,
 'সরকার': 1.0}

Token les moins spécifiques:

{'উত্তর': -0.23552848338828697,
 'কথা': -0.42169597926010005,
 'কলকাতা': -0.9965687665223726,
 'দল': -0.42169597926010005,
 'পরিকল্পনা': -0.23552848338828697,
 'প্রধানমন্ত্রী': -0.6984754075169404,
 'প্রাক্তন': -0.42169597926010005,
 'লড়া': -0.42169597926010005,
 'লোক': -0.23552848338828697,
 'শহর': -0.23552848338828697}


### Conclusion

- Si au niveau extraction et automatisation des collecte de corpus ce projet a pu témoigner la puissance des outils tal déjà existente, niveau sémantique de corpus, il y du travail à faire. 
Après ? nous pouvons constater que... Les approches abordées sont un peu généralistes. 
- une tâche avec complexes avec des moyens informatique et techniques limités.
- sur plusieurs point de vu
    - défintion de ce qu'est un corpus constrastif
    - création [ non - erroné ou non-biasé humainement ou informatiquement ] ce corpus
    - l'élaboration de l'algorithme qui permette d'extaire les caractéristique du corpus (nombre de tokens, nombre de topics à modéliser)
    - encodage extraction des données des fichiers *html*
    - (merci à Mme. WANG pour son assistance )
- demanderait une recherche extensive de ce qu'est un corpus constrastif, les définition , s'il en existe un, et de suivre les pour répérer les essentiels...


### P.S.

Ce rapport se trouve en version `jupyter notebook` dans le dossier `scripts`. 

Normalement, l'installation des bibliothèques mentionnées dans `requirement.txt` permettent d'exécuter le notebook afin de reproduire les résultats. (Il se peut que quelques bibiliothèques inutiles soient listées dans `requirement.txt`). Le package `Textometry` est nécessaire pour les calculs de spécificité. 


### Sitographie
- https://www.lancaster.ac.uk/fss/courses/ling/corpus/Corpus4/4SEMANT.HTM